In [ ]:
%matplotlib inline

In [ ]:
from matplotlib import pyplot


pyplot.style.use('seaborn')

# Eureka!

## Why Eureka?

## Inspiration

In [ ]:
from IPython.display import Image
from IPython.display import SVG


Image(url='images/img_1153.jpg')

## Constant translational acceleration in a straight line

Acceleration is defined as the rate of change of velocity:

$$a = \frac{\mathrm{d}v}{\mathrm{d}t}$$

Velocity (speed) is the rate of change of position:

$$v = \frac{\mathrm{d}x}{\mathrm{d}t}$$

In [ ]:
import sympy
from sympy import symbols, Eq, Integral, solve
from sympy.plotting import plot, plot_parametric, plot_implicit


sympy.init_printing()

In [ ]:
a, x, x0, v, v0, t, t0 = symbols('a x x_0 v v_0 t t_0')

In [ ]:
left = Integral(a, (t, 0, t))
right = Integral(1, (v, v0, v))
equation = Eq(left, right)
equation

In [ ]:
equation = equation.doit()
equation

In [ ]:
v = solve(equation, v)[0]
v

In [ ]:
left = Integral(v, (t, 0, t))
right = Integral(1, (x, x0, x))
equation = Eq(left, right)
equation

In [ ]:
equation = equation.doit()
equation

In [ ]:
solutions = solve(equation, t)
solutions

### Setting initial conditions and acceleration

In [ ]:
s0 = solutions[0].subs({'v_0': 0, 'x_0': 0, 'a': 9.8})
s0

In [ ]:
s1 = solutions[1].subs({'v_0': 0, 'x_0': 0, 'a': 9.8})
s1

In [ ]:
plot(s0, (x, 0, 10), title='Fall duration',
     xlabel='Distance (m)', ylabel='Time (s)')

In [ ]:
s0.subs('x', 13).evalf()

# Experiment

## Environment

In [ ]:
Image(url='images/experiment_rule.jpg', width=300)

In [ ]:
Image(url='images/experiment_height.jpg', width=300)

## Audio analysis

In [ ]:
import os

import numpy
from scipy.signal import resample
from scipy.io import wavfile
from bokeh.plotting import figure, show

from bokeh.io import output_notebook


output_notebook()

In [ ]:
wav = wavfile.read('data/gravity_audio/180.wav')
print(wav)

In [ ]:
from pathlib import Path


data_path = Path('data/gravity_audio')
samples = [wavfile.read(fname)[1]
           for fname in sorted(data_path.glob('*.wav'))]

In [ ]:
def plot_audio(data):
    f = figure(width=800, height=400,
               title='WAV file plot',
               x_axis_label='Time (s)',
               y_axis_label='Amplitude')
    f.line(x=numpy.arange(len(data)) / len(data), y=numpy.array(data))
    show(f)

In [ ]:
plot_audio(samples[0])

In [ ]:
plot_audio(samples[-1])

## Convolution

In [ ]:
window = int(0.02 * len(samples[-1]))

In [ ]:
def convolution(data, window):
    filtered = [0] * len(data)
    for i in range(window - 1, len(data)):
        for j in range(window):
            filtered[i] = max(filtered[i], abs(data[i - j]))
    return filtered

In [ ]:
%time filtered = convolution(samples[-1], window)

In [ ]:
plot_audio(filtered)

### Optimization

In [ ]:
# Numba
import numba


@numba.jit
def convolution_numba(data, window):
    filtered = [0] * len(data)
    for i in range(window - 1, len(data)):
        for j in range(window):
            filtered[i] = max(filtered[i], abs(data[i - j]))
    return filtered


%timeit convolution_numba(samples[-1], window)

In [ ]:
Image(url='images/jurassic.gif', width=600)

In [ ]:
filtered_numba = convolution_numba(samples[-1], window)
plot_audio(filtered_numba)

In [ ]:
filtered == filtered_numba

In [ ]:
@numba.jit
def convolution_optimized(data, window):
    filtered = numpy.zeros(len(data))
    abs_data = numpy.abs(data)
    for i in range(window, len(data)):
        filtered[i] = abs_data[i - window:i].max()
    return filtered


%timeit convolution_optimized(samples[-1], window)

In [ ]:
Image(url='images/obama.gif', width=600)

In [ ]:
filtered_optimized = convolution_optimized(samples[-1], window)
plot_audio(filtered_optimized)

In [ ]:
import pandas

s = pandas.Series(samples[-1])
%timeit s.abs().rolling(window).max()

In [ ]:
plot_audio(s.abs().rolling(window).max())

## Edge detection

In [ ]:
@numba.jit
def edge_detect(data, threshold=30000):
    filtered = numpy.zeros(len(data))
    N = int(0.02 * len(data))
    for i in range(N - 1, len(data)):
        if data[i] <= threshold:
            continue
        filtered[i] = 1.
        for j in range(N - 1):
            if data[i - j - 1] > threshold:
                filtered[i] = 0.
                break
    return filtered

In [ ]:
edge = edge_detect(filtered_optimized)
edge.sum()

In [ ]:
plot_audio(edge)

In [ ]:
def edge_time_diff(edge):
    itemindex = numpy.where(edge==1)
    return (itemindex[0][1] - itemindex[0][0]) / 44100


time_diff = edge_time_diff(edge)
time_diff

## Time

In [ ]:
times = [0]
for data in samples:
    filtered = convolution_optimized(data, window)
    edge = edge_detect(filtered)
    time_diff = edge_time_diff(edge)
    times.append(time_diff)
times = numpy.array(times)

In [ ]:
distances = numpy.arange(len(times)) * 0.05
g_estimations = (2 * distances) / (times ** 2)

g_estimated = g_estimations[1:].mean()
error = (9.8 - g_estimated) / 9.8

g_estimated, error

In [ ]:
names = ['Theoretical', 'Measured']
f = figure(y_range=names, width=800, height=200,
           x_axis_label='Acceleration (m/s²)', title='G estimation')
f.hbar(y=names, height=0.8, right=[9.8, g_estimated])
show(f)

In [ ]:
real_times = [numpy.sqrt((2 * 0.05 * i) / 9.8) for i in range(len(times))]


f = figure(width=800, height=400,
           title='Theoretical versus measured time',
           x_axis_label='Distance (m)',
           y_axis_label='Time (s)')
f.circle(distances, times, fill_color=None, color='red', legend='Measured')
f.circle(distances, real_times, fill_color=None, legend='Theoretical')
f.legend.location = 'bottom_right'
show(f)

We can calculate the maximum absolute error in the time measure (in seconds):

In [ ]:
numpy.abs(real_times - times).max()

In [ ]:
error = times - real_times

f = figure(width=800, height=400,
           title='Measured time error',
           x_axis_label='Distance(m)',
           y_axis_label='Error (ms)')
f.line(distances, error)
show(f)

# Linear regression

$$f(x) = a x + b$$

In [ ]:
x = symbols('x')
line = 1 * x + 0
plot(line, title='Line plot', xlabel='x', ylabel='f(x)')

In [ ]:
from sympy.utilities.lambdify import lambdify


model = lambdify(x, line, 'numpy')
model(distances)

In [ ]:
f = figure(width=800, height=400,
           title='Model approximation',
           x_axis_label='Distance (m)',
           y_axis_label='Time (s)')
f.circle(distances, times, fill_color=None, color='red', legend='Measured')
f.line(distances, model(distances), legend='Model')
f.legend.location = 'bottom_right'
show(f)

A popular way to measure the error of our model is the residual sum of squares:

$$RSS = \sum_{i=1}^{n}{(y_i - \hat{y}_i)^2}$$

In [ ]:
def rss(y, y_hat):
    return numpy.sum((y - y_hat) ** 2)


rss(times, model(distances))

## The minimization/maximization problem

In [ ]:
SVG('images/min-max-problem.svg')

In [ ]:
@numpy.vectorize
def model_rss(a, b):
    y_hat = a * distances + b
    return rss(times, y_hat)


a = numpy.linspace(0, 1, 50)
b = numpy.linspace(0, 1, 50)
agrid, bgrid = numpy.meshgrid(a, b)

errors = model_rss(agrid, bgrid)

In [ ]:
import plotly
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode

import numpy as np


init_notebook_mode(connected=True)

surface = go.Surface(x=a, y=b, z=errors, colorscale='Viridis')
data = [surface]

layout = go.Layout(
    title='RSS curve',
    scene=dict(
        xaxis=dict(
            title='x == a',
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            title='y == b',
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=dict(
            title='z == rss',
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        )
    )
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='RSS curve', show_link=False)

The linear regression problem has a closed-form solution, but it involves calculating the inverse of a matrix, which is an operation with complexity $\mathcal{O}(n^{3})$ or at most $\mathcal{O}(n^{2.373})$.

## Gradient descent

In [ ]:
x = symbols('x')

parabola = x ** 2
plot(parabola, title='Parabola equation')

In [ ]:
from sympy import diff


diff_parabola = diff(parabola, x)
plot(diff_parabola, title='Parabola differentiation')

For convex functions optimum occurs when:

$$
\frac{\mathrm{d}f(x)}{\mathrm{d}x} = 0
$$

So we can converge step by step with:

$$
x^{t + 1} = x - \eta \frac{\mathrm{d}f(x)}{\mathrm{d}x}
$$

Stopping when:

$$
\left| \frac{\mathrm{d}f(x)}{\mathrm{d}x} \right| < \epsilon
$$

In [ ]:
steps = [-9]
lr = 0.1

for _ in range(30):
    before = steps[-1]
    after = before - lr * diff_parabola.subs({'x': before}).evalf()
    steps.append(after)

steps = numpy.array(steps, dtype='float64')

xrange = numpy.linspace(-10, 10, 50)
model = lambdify(x, parabola, 'numpy')

f = figure(width=800, height=400, title='Gradient descent steps')
f.circle(steps, model(steps), fill_color=None, color='red', legend='Steps', size=6)
f.line(steps, model(steps), color='red', legend='Steps')
f.line(xrange, model(xrange), legend='Model')
f.legend.location = 'bottom_right'
show(f)

Sometimes:

$$\eta = \eta(t) = \frac{\eta}{t}$$

# Linear regression with scikit-learn

Simple linear regression model to predict the time:

$$f(x) = a x + b$$

In [ ]:
numpy.random.seed(42)

train_distances = distances[::3]
train_times = times[::3] + numpy.random.randn(train_distances.size) * train_distances * 0.05

df = pandas.DataFrame({'distances': train_distances, 'times': train_times})
df.head()

In [ ]:
f = figure(width=800, height=400,
           title='Reduced set of measures',
           x_axis_label='Distance (m)',
           y_axis_label='Time (s)')
f.circle(x=df['distances'], y=df['times'], fill_color=None, color='red', legend='Measured')
f.legend.location = 'bottom_right'
show(f)

In [ ]:
from sklearn import datasets, linear_model

In [ ]:
# Create linear regression object
model = linear_model.LinearRegression()

# Train the model using the training sets
model.fit(df[['distances']], df['times'])
print(model.intercept_, model.coef_)

In [ ]:
predictions = model.predict(df[['distances']])
predictions

In [ ]:
f = figure(width=800, height=400,
           title='Model approximation',
           x_axis_label='Distance (m)',
           y_axis_label='Time (s)')
f.circle(x=df['distances'], y=df['times'], fill_color=None, color='red', legend='Measured')
f.line(x=df['distances'], y=predictions, legend='Model')
f.legend.location = 'bottom_right'
show(f)

In [ ]:
error = rss(predictions, df['times'])
t_prediction = model.predict(numpy.array(13).reshape(-1, 1))[0]
t_real = s0.subs('x', 13).evalf()

print('RSS: %.4g' % error)
print('Time (pred): %.4g' % t_prediction)
print('Time (real): %.4g' % t_real)
print('Error: %.4g' % (t_prediction - t_real))

In [ ]:
model.predict(numpy.array(13).reshape(-1, 1))

In [ ]:
model.intercept_ + model.coef_ * 13.

## Linear regression (new features)

A new linear regression model using a transformation of the input:

$$f(h(x)) = a h(x) + b$$

In [ ]:
df['sqrt_distances'] = numpy.sqrt(df['distances'])
df = df[['sqrt_distances', 'distances', 'times']]
df.head()

In [ ]:
# Create linear regression object
model = linear_model.LinearRegression()

# Train the model using the training sets
model.fit(df[['sqrt_distances']], df['times'])

linspace = numpy.linspace(0, 2., 100).reshape(-1, 1)
predictions = model.predict(linspace ** 0.5)

In [ ]:
f = figure(width=800, height=400,
           title='Model approximation',
           x_axis_label='Distance (m)',
           y_axis_label='Time (s)')
f.circle(x=df['distances'], y=df['times'], fill_color=None, color='red', legend='Measured')
f.line(x=linspace.T[0], y=predictions.T, legend='Model')
f.legend.location = 'bottom_right'
show(f)

In [ ]:
error = rss(model.predict(df[['sqrt_distances']]), df['times'])
t_prediction = model.predict(numpy.array(13).reshape(-1, 1))[0]
t_real = s0.subs('x', 13).evalf()

print('RSS: %.4g' % error)
print('Time (pred): %.4g' % t_prediction)
print('Time (real): %.4g' % t_real)
print('Error: %.4g' % (t_prediction - t_real))

In [ ]:
model.intercept_ + model.coef_ * 13. ** 0.5

## Another transformation: normalization

## Linear regression (multiple features)

Linear regression model with multiple features:

$$f(x) = w_0 + w_1 x_1 + w_2 x_2$$

In [ ]:
# Create linear regression object
model = linear_model.LinearRegression()

# Train the model using the training sets
model.fit(df[['sqrt_distances', 'distances']], df['times'])

linspace = numpy.linspace(0, 2., 100).reshape(-1, 1)
predictions = model.predict(numpy.column_stack([linspace ** 0.5, linspace]))

In [ ]:
f = figure(width=800, height=400,
           title='Model approximation',
           x_axis_label='Distance (m)',
           y_axis_label='Time (s)')
f.circle(x=df['distances'], y=df['times'], fill_color=None, color='red', legend='Measured')
f.line(x=linspace.T[0], y=predictions.T, legend='Model')
f.legend.location = 'bottom_right'
show(f)

In [ ]:
print(model.intercept_, model.coef_)

In [ ]:
error = rss(model.predict(df[['sqrt_distances', 'distances']]), df['times'])
t_prediction = model.predict(numpy.array([numpy.sqrt(13.), 13]).reshape(1, -1))[0]
t_real = s0.subs('x', 13).evalf()

print('RSS: %.4g' % error)
print('Time (pred): %.4g' % t_prediction)
print('Time (real): %.4g' % t_real)
print('Error: %.4g' % (t_prediction - t_real))

In [ ]:
model.intercept_ + model.coef_[0] * numpy.sqrt(13.) + model.coef_[1] * 13.

## Linear regression (many features)

Linear regression model with many features:

$$f(x) = w_0 + w_1 x_1 + w_2 x_2 + ... + w_N x_N$$

In [ ]:
Image('images/machine_learning.png')

In [ ]:
for i in range(2, 20):
    df['distances_%02d' % i] = df['distances'] ** i
df.drop('times', axis=1).head()

In [ ]:
# Create linear regression object
model = linear_model.LinearRegression()

# Train the model using the training sets
model.fit(df.drop('times', axis=1), df['times'])

from sklearn.preprocessing import PolynomialFeatures
linspace = numpy.linspace(0, 2., 100).reshape(-1, 1)
poly = PolynomialFeatures(degree=19, include_bias=False).fit_transform(linspace)
predictions = model.predict(numpy.column_stack([linspace ** 0.5, poly]))

In [ ]:
error = rss(model.predict(df.drop('times', axis=1)), df['times'])

print('RSS: %.4g' % error)

In [ ]:
f = figure(width=800, height=400,
           title='Model approximation',
           x_axis_label='Distance (m)',
           y_axis_label='Time (s)')
f.circle(x=df['distances'], y=df['times'], fill_color=None, color='red', legend='Measured')
f.line(x=linspace.T[0], y=predictions.T, legend='Model')
show(f)

In [ ]:
error = rss(model.predict(df.drop('times', axis=1)), df['times'])
poly = PolynomialFeatures(degree=19, include_bias=False).fit_transform(numpy.array([13.]).reshape(-1, 1))
t_prediction = model.predict(numpy.column_stack([numpy.array([13.]) ** 0.5, poly]))[0]
t_real = s0.subs('x', 13).evalf()

print('RSS: %.4g' % error)
print('Time (pred): %.4g' % t_prediction)
print('Time (real): %.4g' % t_real)
print('Error: %.4g' % (t_prediction - t_real))

# Regularization

Regularization is about imposing a penalty on the size of coefficients, to address some of the problems of Ordinary Least Squares.

# Norms

Norms are a way to measure the size of a vector.

### Euclidean norm

A distance between 2 points can be calculated as:

$$
d(p, q) = \sqrt{|q_1 - p_1|^2 + |q_2 - p_2|^2 + ... + |q_n - p_n|^2}
$$

In [ ]:
f = figure(width=400, height=400, match_aspect=True,
           title='Reduced set of measures',
           x_axis_label='x1',
           y_axis_label='x2')
f.circle(x=(1, 2), y=(3, 1), fill_color=None, color='red')
f.line(x=(1, 2), y=(3, 1))
show(f)

In [ ]:
numpy.sqrt(abs(2 - 1) ** 2 + abs(1 - 3) ** 2)

The norm is simply the distance between a point and the origin:

$$
\left\| p \right\|_2 = \sqrt{|p_1|^2 + |p_2|^2 + ... + |p_n|^2}
$$

In [ ]:
f = figure(width=400, height=400, match_aspect=True,
           title='Reduced set of measures',
           x_axis_label='x1',
           y_axis_label='x2')
f.circle(x=(1, 2), y=(3, 1), fill_color=None, color='red')
f.asterisk(x=0, y=0, fill_color=None, color='red', size=10)
f.line(x=(0, 1), y=(0, 3))
f.line(x=(0, 2), y=(0, 1))
show(f)

In [ ]:
numpy.sqrt(1 ** 2 + 3 ** 2), numpy.sqrt(2 ** 2 + 1 ** 2)

### Manhattan norm

A distance between 2 points can also be calculated as:

$$
d(p, q) = |q_1 - p_1| + |q_2 - p_2| + ... + |q_n - p_n|
$$

In [ ]:
f = figure(width=400, height=400, match_aspect=True,
           title='Reduced set of measures',
           x_axis_label='x1',
           y_axis_label='x2')
f.circle(x=(1, 2), y=(3, 1), fill_color=None, color='red')
f.line(x=(1, 2), y=(1, 1))
f.line(x=(1, 1), y=(1, 3))
show(f)

In [ ]:
abs(2 - 1) + abs(1 - 3)

In [ ]:
f = figure(width=400, height=400, match_aspect=True,
           title='Reduced set of measures',
           x_axis_label='x1',
           y_axis_label='x2')
f.circle(x=(1, 2), y=(3, 1), fill_color=None, color='red')
f.line(x=(2, 2), y=(1, 2))
f.line(x=(2, 1), y=(2, 2))
f.line(x=(1, 1), y=(2, 3))
show(f)

The norm is, again, the distance between a point and the origin:

$$
\left\| p \right\|_1 = |p_1| + |p_2| + ... + |p_n|
$$

In [ ]:
f = figure(width=400, height=400, match_aspect=True,
           title='Reduced set of measures',
           x_axis_label='x1',
           y_axis_label='x2')
f.circle(x=(1, 2), y=(3, 1), fill_color=None, color='red')
f.asterisk(x=0, y=0, fill_color=None, color='red', size=10)
f.line(x=(0, 2), y=(0, 0))
f.line(x=(2, 2), y=(0, 1))
f.line(x=(0, 0), y=(0, 3))
f.line(x=(0, 1), y=(3, 3))
show(f)

## Ridge regression (L2 regularization) 

Later we used some:

$$\min_{w} \left\| y - \hat{y} \right\| ^2_2 = \min_{w} \left\| y - Xw \right\| ^2_2$$

Now we will penalize higher parameters by taking into account the size of the $w$ vector:

$$
\min_{w} \left(\left\| y - Xw \right\| ^2_2 + \alpha \left\|w\right\|^2_2\right)
$$

In [ ]:
# Create linear regression object
model = linear_model.Ridge(alpha=0.5, fit_intercept=False)

# Train the model using the training sets
model.fit(df.drop('times', axis=1), df['times'])
print(model.intercept_, model.coef_)

In [ ]:
linspace = numpy.linspace(0, 2., 100).reshape(-1, 1)
poly = PolynomialFeatures(degree=19, include_bias=False).fit_transform(linspace)
predictions = model.predict(numpy.column_stack([linspace ** 0.5, poly]))

In [ ]:
f = figure(width=800, height=400,
           title='Model approximation',
           x_axis_label='Distance (m)',
           y_axis_label='Time (s)')
f.circle(x=df['distances'], y=df['times'], fill_color=None, color='red', legend='Measured')
f.line(x=linspace.T[0], y=predictions.T, legend='Model')
f.legend.location = 'bottom_right'
show(f)

In [ ]:
error = rss(model.predict(df.drop('times', axis=1)), df['times'])
poly = PolynomialFeatures(degree=19, include_bias=False).fit_transform(numpy.array([13.]).reshape(-1, 1))
t_prediction = model.predict(numpy.column_stack([numpy.array([13.]) ** 0.5, poly]))[0]
t_real = s0.subs('x', 13).evalf()

print('RSS: %.4g' % error)
print('Time (pred): %.4g' % t_prediction)
print('Time (real): %.4g' % t_real)
print('Error: %.4g' % (t_prediction - t_real))

## Lasso regression (L1 regularization)

$$
\min_{w} \left(\left\| y - Xw \right\| ^2_2 + \alpha \left\|w\right\|^2_1\right)
$$

In [ ]:
# Create linear regression object
model = linear_model.Lasso(alpha=0.005, max_iter=1e7, fit_intercept=False)

# Train the model using the training sets
model.fit(df.drop('times', axis=1), df['times'])
print(model.intercept_, model.coef_)

In [ ]:
linspace = numpy.linspace(0, 2., 100).reshape(-1, 1)
poly = PolynomialFeatures(degree=19, include_bias=False).fit_transform(linspace)
predictions = model.predict(numpy.column_stack([linspace ** 0.5, poly]))

In [ ]:
f = figure(width=800, height=400,
           title='Model approximation',
           x_axis_label='Distance (m)',
           y_axis_label='Time (s)')
f.circle(x=df['distances'], y=df['times'], fill_color=None, color='red', legend='Measured')
f.line(x=linspace.T[0], y=predictions.T, legend='Model')
f.legend.location = 'bottom_right'
show(f)

In [ ]:
error = rss(model.predict(df.drop('times', axis=1)), df['times'])
poly = PolynomialFeatures(degree=19, include_bias=False).fit_transform(numpy.array([13.]).reshape(-1, 1))
t_prediction = model.predict(numpy.column_stack([numpy.array([13.]) ** 0.5, poly]))[0]
t_real = s0.subs('x', 13).evalf()

print('RSS: %.4g' % error)
print('Time (pred): %.4g' % t_prediction)
print('Time (real): %.4g' % t_real)
print('Error: %.4g' % (t_prediction - t_real))

## Ridge vs. Lasso (visualization)

# Validation set